In [1]:
import rasterio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling
from rasterio.crs import CRS
from pyproj import Transformer
import numpy as np
from scipy.ndimage import generic_filter
from scipy.stats import mode
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sqlalchemy import create_engine
import psycopg2

In [2]:
url = r'https://www.mrlc.gov/downloads/sciweb1/shared/mrlc/data-bundles/Annual_NLCD_LndCov_2023_CU_C1V0.tif'

In [3]:
with rasterio.open(url) as src:
    sr = src.crs

# Define the bounding box in EPSG:4326 (min_x, min_y, max_x, max_y)
bounding_box_epsg4326 = (-97.5, 43, -89, 49.5)

# Define the target projection string (AEA based on WGS84)
#target_crs = (
#    "+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 "
#    "+x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"
#)
target_crs = sr

# Initialize the Transformer for conversion
transformer = Transformer.from_crs("EPSG:4326", target_crs, always_xy=True)

# Transform the bounding box coordinates
min_x, min_y = transformer.transform(bounding_box_epsg4326[0], bounding_box_epsg4326[1])
max_x, max_y = transformer.transform(bounding_box_epsg4326[2], bounding_box_epsg4326[3])

# New bounding box in the target CRS
bounding_box_aea = (min_x, min_y, max_x, max_y)
print("Bounding box in the target CRS:", bounding_box_aea)

Bounding box in the target CRS: (-121629.82955944678, 2223662.3251429657, 514102.3021655515, 2962023.2392924265)


In [4]:
# Open the raster from a URL
with rasterio.open(url) as src:
    # Step 1: Create a window using the bounding box
    window = from_bounds(*bounding_box_aea, transform = src.transform)
    
    # Step 2: Read the clipped raster data
    clipped_raster = src.read(1, window = window)
    
    # Step 3: Update the transform for the clipped raster
    clipped_transform = src.window_transform(window)
    
    # Step 4: Update the profile for the clipped raster
    profile = src.profile
    profile.update({
        "height": clipped_raster.shape[0],
        "width": clipped_raster.shape[1],
        "transform": clipped_transform
    })
    
    # Step 5: Define the target resolution (1 km = 1000 meters)
    target_resolution = 1000  # in meters
    scale_factor = target_resolution / src.res[0]  # Assuming square pixels
    new_width = int(clipped_raster.shape[1] / scale_factor)
    new_height = int(clipped_raster.shape[0] / scale_factor)
    
    # Step 6: Resample the raster
    resampled_raster = src.read(
        1,
        window = window,
        out_shape = (new_height, new_width),
        resampling = Resampling.average  # Use average, or mode for categorical data
    )
    
    # Step 7: Update the transform for the resampled raster
    resampled_transform = clipped_transform * clipped_transform.scale(
        clipped_raster.shape[1] / new_width,
        clipped_raster.shape[0] / new_height
    )
    
    # Step 8: Update profile for the resampled raster
    profile.update({
        "height": new_height,
        "width": new_width,
        "transform": resampled_transform,
        "dtype": rasterio.float32  # Set to float for NaN handling
    })

In [5]:
# Define the custom function for the kernel
def process_kernel(window):
    # The window is a flattened array of kernel values
    center_value = window[len(window) // 2]  # Get the center pixel value
    unique_values, counts = np.unique(window, return_counts=True)
    
    # Check if the center pixel is unique
    if counts[np.where(unique_values == center_value)[0][0]] == 1:
        # If unique, replace with the most common value in the kernel
        common_value = mode(window, nan_policy="omit").mode[0]
        return common_value
    else:
        # If not unique, leave it unchanged
        return center_value

# Define the kernel size (e.g., 3x3 moving window)
kernel_size = 3

# Apply the filter
corrected_raster = generic_filter(
    resampled_raster,
    function=process_kernel,
    size=kernel_size,
    mode='constant',
    cval=np.nan  # Optional: Edge pixels treated as NaN
)

C:\Users\tjjoh\AppData\Local\Temp\ipykernel_10640\3482037650.py:10: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  common_value = mode(window, nan_policy="omit").mode[0]


In [6]:
valid_values = {11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95}
corrected_raster = corrected_raster.astype(float)  # Convert to float for NaN support
corrected_raster[~np.isin(corrected_raster, list(valid_values))] = np.nan

In [7]:
# Prepare lists for point coordinates and values
points = []
values = []

# Loop through each cell in the raster
rows, cols = corrected_raster.shape
for row in range(rows):
    for col in range(cols):
        value = corrected_raster[row, col]
        if not np.isnan(value):  # Skip no-data values
            # Calculate the coordinates of the cell's center
            x, y = rasterio.transform.xy(profile['transform'], row, col, offset="center")
            points.append(Point(x, y))  # Add the point
            values.append(value)        # Add the corresponding value

# Create a GeoDataFrame
landcover = gpd.GeoDataFrame({"Land Cover Classification": values, "geometry": points}, crs = src.crs)
landcover = landcover.to_crs('EPSG:4326')

In [8]:
#connect to the new database to enable PostGIS
connection_string = f'postgresql://<user>:<password>@34.133.43.30:5432/lab2'
engine = create_engine(connection_string)

# Push the GeoDataFrame to PostGIS
table_name = "mn_landcover"
landcover.to_postgis(table_name, engine, if_exists="replace", index=False)

print(f"GeoDataFrame successfully pushed to the PostGIS table '{table_name}'.")

GeoDataFrame successfully pushed to the PostGIS table 'mn_landcover'.
